In [ ]:
%pip install pandas
%pip install matplotlib
%pip install seaborn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import os
from pathlib import Path

In [ ]:
sns.set(style="whitegrid")

# Grab all result CSV files
files = glob.glob("../results/*.csv")

records = []

for filepath in files:
    filename = Path(filepath).stem  # e.g., basic_dmalloc_amount1000_size1

    # Extract metadata from filename
    match = re.match(r"(?P<benchmark>\w+)_(?P<allocator>\w+)_amount(?P<amount>\d+)(?:_size(?P<size>\d+))?", filename)
    if not match:
        print(f"Skipping unrecognized file: {filename}")
        continue

    meta = match.groupdict()
    benchmark = meta["benchmark"]
    allocator = meta["allocator"]
    amount = int(meta["amount"])
    size = int(meta["size"]) if meta["size"] else None

    # Read CSV, skip command row
    df = pd.read_csv(filepath)

    row = df.iloc[0]  # Only one row per file

    records.append({
        "benchmark": benchmark,
        "allocator": allocator,
        "amount": amount,
        "size": size,
        "mean": float(row["mean"]),
        "stddev": float(row["stddev"]),
        "min": float(row["min"]),
        "max": float(row["max"]),
    })

# Create DataFrame
df = pd.DataFrame(records)

# For plotting: treat missing sizes as a separate case (e.g. varying_allocs)
df["size"] = df["size"].fillna(-1)

In [ ]:
import os
from datetime import datetime

# Set up the output directory
today = datetime.now().strftime("%Y-%m-%d")
output_dir = f"./benchmark_results/{today}"
os.makedirs(output_dir, exist_ok=True)

allocators = df["allocator"].unique()
palette = sns.color_palette("tab10", n_colors=len(allocators))
color_mapping = dict(zip(sorted(allocators), palette))

for benchmark in df["benchmark"].unique():
    bench_df = df[df["benchmark"] == benchmark]

    for size in sorted(bench_df["size"].unique()):
        size_df = bench_df[bench_df["size"] == size]

        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=size_df,
            x="amount",
            y="mean",
            hue="allocator",
            marker="o",
            palette=color_mapping,
        )

        title_size = " (varying sizes)" if size == -1 else f" (size={size} bytes)"
        plt.title(f"{benchmark}{title_size}")
        plt.xlabel("Amount of Allocations")
        plt.ylabel("Mean Time (s)")
        plt.legend(title="Allocator")
        plt.tight_layout()

        # Prepare filename-safe benchmark name
        safe_benchmark = benchmark.replace("/", "_").replace(" ", "_")
        size_str = "varying" if size == -1 else str(size)
        file_name = f"{safe_benchmark}_size_{size_str}.png"
        file_path = os.path.join(output_dir, file_name)

        # Save the figure
        plt.savefig(file_path)
        plt.close()


NameError: name 'df' is not defined

In [ ]:
import os
from datetime import datetime

# Set up the base output directory
today = datetime.now().strftime("%Y-%m-%d")
base_output_dir = f"./benchmark_results/{today}"

allocators = df["allocator"].unique()
palette = sns.color_palette("tab10", n_colors=len(allocators))
color_mapping = dict(zip(sorted(allocators), palette))

for benchmark in df["benchmark"].unique():
    bench_df = df[df["benchmark"] == benchmark]

    # Make filename-safe benchmark name for folder
    safe_benchmark = benchmark.replace("/", "_").replace(" ", "_")
    benchmark_dir = os.path.join(base_output_dir, safe_benchmark)
    os.makedirs(benchmark_dir, exist_ok=True)

    for size in sorted(bench_df["size"].unique()):
        size_df = bench_df[bench_df["size"] == size]

        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=size_df,
            x="amount",
            y="mean",
            hue="allocator",
            marker="o",
            palette=color_mapping,
        )

        title_size = " (varying sizes)" if size == -1 else f" (size={size} bytes)"
        plt.title(f"{benchmark}{title_size}")
        plt.xlabel("Amount of Allocations")
        plt.ylabel("Mean Time (s)")
        plt.legend(title="Allocator")
        plt.tight_layout()

        size_str = "varying" if size == -1 else str(size)
        file_name = f"size_{size_str}.png"
        file_path = os.path.join(benchmark_dir, file_name)

        plt.savefig(file_path)
        plt.close()
